In [1]:
import os
os.chdir('/home/potusvn/Projects/kickstarter_prediction')

In [2]:
import pandas as pd
import pickle
import numpy as np

from os.path import join
from settings import *

In [3]:
df = pd.read_csv('data/raw/ks-projects-201801.csv', encoding='latin1', low_memory=False)

In [4]:
df = df[(df.state == 'successful') | (df.state == 'failed')]

In [5]:
df['success'] = np.where(df.state == 'successful', 1, 0)

In [6]:
df.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real', 'usd_goal_real', 'success'],
      dtype='object')

In [7]:
df = df.drop(['ID', 'pledged', 'usd pledged','state', 'goal'], axis=1)

In [8]:
df.loc[:, 'usd_pledged_real'] = pd.to_numeric(df['usd_pledged_real'], downcast='float', errors='coerce')
df.loc[:, 'usd_goal_real'] = pd.to_numeric(df['usd_goal_real'], downcast='float', errors='coerce')
df.loc[:, 'backers'] = pd.to_numeric(df.backers, errors='raise', downcast='integer')

# Convert launched, and deadline to datetime objects
for col in ['launched', 'deadline']:
    df.loc[:, col] = pd.to_datetime(df[col], errors='coerce')

In [9]:
features = df.copy()

# Filter out the null name
features = features[features.name.notna()]


features['name_length'] = df.name.str.len()

features['duration'] = (df.deadline - df.launched).dt.days + 1

In [10]:
new_dataset = features[['backers', 'usd_goal_real', 'name_length','duration', 'main_category', 'country', 'success']]

In [11]:
new_dataset = pd.get_dummies(new_dataset, columns = ['main_category', 'country'], dummy_na=False, drop_first=True)


In [12]:
new_dataset.columns

Index(['backers', 'usd_goal_real', 'name_length', 'duration', 'success',
       'main_category_Comics', 'main_category_Crafts', 'main_category_Dance',
       'main_category_Design', 'main_category_Fashion',
       'main_category_Film & Video', 'main_category_Food',
       'main_category_Games', 'main_category_Journalism',
       'main_category_Music', 'main_category_Photography',
       'main_category_Publishing', 'main_category_Technology',
       'main_category_Theater', 'country_AU', 'country_BE', 'country_CA',
       'country_CH', 'country_DE', 'country_DK', 'country_ES', 'country_FR',
       'country_GB', 'country_HK', 'country_IE', 'country_IT', 'country_JP',
       'country_LU', 'country_MX', 'country_N,0"', 'country_NL', 'country_NO',
       'country_NZ', 'country_SE', 'country_SG', 'country_US'],
      dtype='object')

In [13]:
X = new_dataset.drop(['success'], axis=1)
y = new_dataset.success

In [14]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=0)

In [ ]:
# train_x = train.drop(['success'], axis=1)
# train_y = train.success

# test_x = test.drop(['success'], axis=1)
# test_y = test.success

In [17]:
train_x.head(10)

,backers,usd_goal_real,name_length,duration,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
323096,25,5000.000000,60.0,30,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
136074,3,18000.000000,16.0,35,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14614,103,9863.509766,15.0,24,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
340105,0,6000.000000,32.0,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
100164,25,1000.000000,41.0,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24270,36,1500.000000,34.0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
125383,30,1500.000000,46.0,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12008,164,5000.000000,60.0,83,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
306032,51,15000.000000,47.0,28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
109995,6,50000.000000,53.0,60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [34]:
import tensorflow as tf
from keras import layers, optimizers
from keras import models

model = models.Sequential()
# Input - Layer
model.add(layers.Dense(180, activation="relu", input_shape = (40,)))
# Hidden - Layers
model.add(layers.Dense(80, activation="relu"))
# Output- Layer
model.add(layers.Dense(1, activation=tf.nn.sigmoid))
model.summary()
# compiling the model
model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
results = model.fit(
    train_x, train_y,
    epochs=100,
    batch_size=180,
    validation_data=(test_x, test_y)
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 180)               7380      
_________________________________________________________________
dense_25 (Dense)             (None, 80)                14480     
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 81        
Total params: 21,941
Trainable params: 21,941
Non-trainable params: 0
_________________________________________________________________
Train on 265337 samples, validate on 66335 samples
Epoch 1/100
265337/265337 [==============================] - 6s 21us/step - loss: 8.8971 - acc: 0.4408 - val_loss: 9.5493 - val_acc: 0.4010
Epoch 2/100
265337/265337 [==============================] - 6s 21us/step - loss: 9.4921 - acc: 0.4046 - val_loss: 9.5493 - val_acc: 0.4010
Epoch 3/100
265337/265337 [==============================] - 5s 20us/st

265337/265337 [==============================] - 5s 20us/step - loss: 0.9886 - acc: 0.9178 - val_loss: 0.8762 - val_acc: 0.9011
Epoch 54/100
265337/265337 [==============================] - 5s 19us/step - loss: 0.6735 - acc: 0.9096 - val_loss: 0.6879 - val_acc: 0.9169
Epoch 55/100
265337/265337 [==============================] - 5s 20us/step - loss: 0.4675 - acc: 0.9133 - val_loss: 0.3546 - val_acc: 0.9300
Epoch 56/100
265337/265337 [==============================] - 5s 20us/step - loss: 0.4090 - acc: 0.9149 - val_loss: 0.3458 - val_acc: 0.9258
Epoch 57/100
265337/265337 [==============================] - 5s 20us/step - loss: 0.3506 - acc: 0.9180 - val_loss: 0.3182 - val_acc: 0.8970
Epoch 58/100
265337/265337 [==============================] - 5s 21us/step - loss: 0.3278 - acc: 0.9183 - val_loss: 0.2845 - val_acc: 0.9305
Epoch 59/100
265337/265337 [==============================] - 5s 21us/step - loss: 0.3091 - acc: 0.9200 - val_loss: 0.2972 - val_acc: 0.9124
Epoch 60/100
265337/265337

## Remove backers

In [35]:
new_dataset2 = features[['usd_goal_real', 'name_length','duration', 'main_category', 'country', 'success']]

In [36]:
new_dataset2 = pd.get_dummies(new_dataset2, columns = ['main_category', 'country'], dummy_na=False, drop_first=True)


In [37]:
X = new_dataset2.drop(['success'], axis=1)
y = new_dataset2.success

In [38]:
from sklearn.model_selection import train_test_split
train_x2, test_x2, train_y2, test_y2 = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=0)

In [39]:
train_x2.shape

(265337, 39)

In [40]:
import tensorflow as tf
from keras import layers, optimizers
from keras import models

model = models.Sequential()
# Input - Layer
model.add(layers.Dense(180, activation="relu", input_shape = (39,)))
# Hidden - Layers
model.add(layers.Dense(80, activation="relu"))
# Output- Layer
model.add(layers.Dense(1, activation=tf.nn.sigmoid))
model.summary()
# compiling the model
model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
results = model.fit(
    train_x2, train_y2,
    epochs=100,
    batch_size=180,
    validation_data=(test_x2, test_y2)
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 180)               7200      
_________________________________________________________________
dense_28 (Dense)             (None, 80)                14480     
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 81        
Total params: 21,761
Trainable params: 21,761
Non-trainable params: 0
_________________________________________________________________
Train on 265337 samples, validate on 66335 samples
Epoch 1/100
265337/265337 [==============================] - 6s 22us/step - loss: 7.9463 - acc: 0.5018 - val_loss: 6.4635 - val_acc: 0.5990
Epoch 2/100
265337/265337 [==============================] - 5s 20us/step - loss: 7.4848 - acc: 0.5306 - val_loss: 9.5068 - val_acc: 0.4011
Epoch 3/100
265337/265337 [==============================] - 6s 21us/st

265337/265337 [==============================] - 6s 21us/step - loss: 0.6751 - acc: 0.5954 - val_loss: 0.6735 - val_acc: 0.5990
Epoch 54/100
265337/265337 [==============================] - 6s 21us/step - loss: 0.6750 - acc: 0.5954 - val_loss: 0.6735 - val_acc: 0.5990
Epoch 55/100
265337/265337 [==============================] - 5s 21us/step - loss: 0.6750 - acc: 0.5954 - val_loss: 0.6735 - val_acc: 0.5990
Epoch 56/100
265337/265337 [==============================] - 5s 20us/step - loss: 0.6750 - acc: 0.5954 - val_loss: 0.6735 - val_acc: 0.5990
Epoch 57/100
265337/265337 [==============================] - 5s 20us/step - loss: 0.6750 - acc: 0.5954 - val_loss: 0.6734 - val_acc: 0.5990
Epoch 58/100
265337/265337 [==============================] - 5s 21us/step - loss: 0.6750 - acc: 0.5954 - val_loss: 0.6736 - val_acc: 0.5990
Epoch 59/100
265337/265337 [==============================] - 5s 20us/step - loss: 0.6750 - acc: 0.5954 - val_loss: 0.6736 - val_acc: 0.5990
Epoch 60/100
265337/265337

In [43]:
np.max(results.history['val_acc'])

0.6059998516095847